<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_02_Aprendizado_Supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie_tic.png' width="600">


# **Aprendizado supervisionado**

Em aprendizado de máquina, podemos distinguir os métodos entre supervisionados e não-supervisionados. Nesta primeira parte da disciplina, abordaremos os métodos de aprendizado supervisionado.

No contexto supervisionado, a estrutura do aprendizado de máquina consiste em estabelecer uma relação entre dados de entrada (informações sobre clientes, por exemplo) e dados de saída (se os clientes contrataram ou não um serviço, por exemplo). Ou seja, para cada dado de entrada, há um rótulo associado a ele. O objetivo, então, é que a máquina aprenda essa relação e consiga estimar com precisão qual seria a saída para novos dados de entrada.


# Matematicamente, ...

Seja $\mathbf{X}$ uma matriz de dados de entrada com $n$ linhas (amostras) e $m$ colunas (atributos ou variáveis de entrada). Seja, também, $\mathbf{x}_i$, $i=1, \ldots, n$, uma linha da matrix $\mathbf{X}$, a qual representa um conjunto de atributos associado a $i$-ésima amostra dessa base de dados. Esse cenário pode representar, por exemplo, um conjunto de informações de clientes de um banco, sendo que cada linha $\mathbf{x}_i$ indicaria caracaterísticas de um cliente, tais como idade, gênero, se possui imóvel ou não, saldo na conta corrente, saldo de investimentos, renda mensal, etc.

Associado à matriz $\mathbf{X}$, na aprendizagem supervisionada, temos então o vetor de saída $\mathbf{y}$, também com $n$ elementos, onde cada elemento $y_i$ indica a saída (rótulo ou variável de saída) da amostra $\mathbf{x}_i$. Em nosso exemplo, $y_i$ poderia indicar o aporte mensal que o cliente faz em investimentos ou se o cliente contratou (ou não) um determinado serviço oferecido pelo banco.

Note que, no exemplo mencionado, o primeiro caso contém uma saída numérica (em reais, por exemplo) e o segundo caso contém uma saída que representa uma classificação (sim ou não). Embora para cada um desses casos temos algoritmos específicos para construir a inteligência computacional, a ideia em termos matemáticos é parecida. O objetivo é encontrar um modelo (ou função) $f(\mathbf{x}_i) = \hat{y}_i$ de tal forma que a saída desta função ($\hat{y}_i$) seja o mais próximo possível da saída verdadeira $y_i$ para todo o $i$ dos dados coletados. Ou seja, via de regra, queremos

$$\min_{\theta} \sum_{i=1}^n \left( \hat{y}_i - \hat{y} \right)^2, $$

onde $\theta$ representa o conjunto de parâmetros do modelo $f(\cdot)$. Por hora, abordaremos os problemas de classificação. Ou seja, problemas onde a saída do modelo é binária (0 ou 1, sim ou não, etc...)

# Exemplo

Queremos enterder os perfis de clientes de um determinado banco para conduzir uma campanha de marketing para um determinado produto. Para isso, a ideia é avaliar, com base em dados de clientes desse banco, se no passado eles já contrataram tal produto (ou similar). A primeira etapa, então, foi coletar os dados dos clientes, assim como a informação se eles contrataram ou não o produto em uma campanha anterior. Os dados podem ser lidos com o comando da sélula seguinte.

*Obs: esses dados foram retirados da base [Bank Marketing](https://archive.ics.uci.edu/dataset/222/bank+marketing)*

In [2]:
import pandas as pd # Importando a biblioteca Pandas

dados = pd.read_csv("https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_bank_marketing.csv", sep = ';') # Lendo os dados
dados.head() # Visualizando as primeiras linhas

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Informações gerais sobre o conjunto de dados:

- age (numeric)
- job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services")
- marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
- education (categorical: "unknown","secondary","primary","tertiary")
- default: has credit in default? (binary: "yes","no")
- balance: average yearly balance, in euros (numeric)
- housing: has housing loan? (binary: "yes","no")
- loan: has personal loan? (binary: "yes","no")

Informações relacionadas ao último contato realizado durante a campanha de marketing:

- contact: contact communication type (categorical: "unknown","telephone",- day: last contact day of the month (numeric)
- month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
- duration: last contact duration, in seconds (numeric)
   
Outras informações:

- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

Variável de saída:

- y - has the client subscribed a term deposit? (binary: "yes","no")

In [4]:
dados.shape

(45211, 17)

Note que nesta base de dados temos 17 informações sobre 45211 clientes, sendo que a última coluna indica a classe à qual o cliente pertence (se contratou ou não o produto). Então, as primeiras 16 colunas dessa base de dados seriam os dados de entrada $\mathbf{X}$ e a última coluna o vetor de classes $\mathbf{y}$.

Na construção de modelos de aprendizado de máquina usando o Python, geralmente separamos esses dados em objetos diferentes. Veja na célula seguinte como fazemos isso.

In [6]:
# Separando os dados de entrada e saída

X = dados.drop('y', axis=1)
y = dados.y

In [8]:
# Visualizando os dados de entrada

X.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown


In [9]:
# Visualizando os dados de saída

y.head()

,y
0,no
1,no
2,no
3,no
4,no


Pronto! Agora temos os dados de entrada e saída separados para criar o modelo de aprendizado de máquina. Mas antes disso, é importante pensar na capacidade de generalização desse modelo a ser construído!

# Treinamento e capacidade de generalização

De posse dos dados de entrada e saída, poderíamos usar todos eles para criar o modelo. Em outras palavras, poderíamos fornecer toda a matriz $\mathbf{X}$ e todo o vetor $\mathbf{y}$ para a máquina aprender. No entanto, ao fazer isso e criar o modelo computacional, não teríamos como avaliar se esse modelo consegue ter bom desempenho em novas amostras, que não foram usadas na construção do modelo.

Sendo assim, é comum ao desenvolver um modelo de aprendiado de máquina a separar os dados entre treinamento e teste. Os dados de treinamento serão utilizados para construir (treinar) a máquina. Feito isso, aplicamos o modelo treinado nos dados de teste, para avaliar a qualidade do mesmo.

Há um comando específico em Python para essa tarefa, o qual faz essa separação de uma forma aleatória. Esse comando se encontra na bilioteca ``sklearn`` (abreviação para Scikit-learn) e é chamado ``train_test_split``. Veja na sequência como usá-lo.

*Obs: a biblioteca ``sklearn`` contém diversos modelos de aprendizado de máquina (dentre outras coisas). Portanto, usaremos bastante essa biblioteca nesta disciplina!*

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(36168, 16) (36168,)
(9043, 16) (9043,)


Note que os argumentos da função ``train_test_split`` são (nessa ordem): os dados de entrada $\mathbf{X}$, os dados de saída $\mathbf{y}$ e a proporção dos dados que serão alocados para a etapa de teste. Nesse exemplo acima, 20\% dos dados serão usados para testar a qualidade do modelo treinado.

Há também outros argumentos que podem ser usados nessa função. Um deles é definir uma semente para "tirar" a aleatoriedade na separação dos dados entre treinamento e teste. Para isso, incluímos o parâmetro ``random_state`` como argumento da função e definimos a semente desejada. Outra customização seria garantir que a mesma proporção das classes nos dados originais $\mathbf{y}$ se mantivesse nos dados de treinamento e teste. Por exemplo, se 40% dos clientes no exemplo contrataram o serviço, podemos impor que, tanto nos dados de teste quanto nos dados de treinamento, haja também 40% de clientes que contrataram o serviço. Para isso, utilizamos o parâmetro ``stratify`` sobre o vetor $\mathbf{y}$. Essa estratificação é necessária quando há uma desbalanceamento entre as classes no vetor $\mathbf{y}$, sendo importante garantir o mesmo cenário em ambos os dados de treinamento e teste para refletir a realidade dos dados. Veja como ficam essas costumizações:

In [18]:
# Antes da estratificação

print("Proporção de sim nos dados originais:", sum(y=='yes')/len(y))
print("Proporção de sim nos dados de treinamento:", sum(y_train=='yes')/len(y_train))
print("Proporção de sim nos dados teste:", sum(y_test=='yes')/len(y_test))

Proporção de sim nos dados originais: 0.11698480458295547
Proporção de sim nos dados de treinamento: 0.11678832116788321
Proporção de sim nos dados teste: 0.11777065133252239


In [19]:
# Após a estratificação

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7, stratify = y)

print("Proporção de sim nos dados originais:", sum(y=='yes')/len(y))
print("Proporção de sim nos dados de treinamento:", sum(y_train=='yes')/len(y_train))
print("Proporção de sim nos dados teste:", sum(y_test=='yes')/len(y_test))

Proporção de sim nos dados originais: 0.11698480458295547
Proporção de sim nos dados de treinamento: 0.11698186241981863
Proporção de sim nos dados teste: 0.11699657193409267


Após usarmos o ``train_test_split`` temos, então, os dados de treinamento $\mathbf{X}_{train}$, $\mathbf{y}_{train}$ e os dados de teste $\mathbf{X}_{test}$, $\mathbf{y}_{test}$ (poderia ser qualquer nome, claro). Os dados de treinamento serão usados exclusivamente para criar o modelo e os dados de teste exclusivamente para avaliar a qualidade do mesmo. E qual a importância de avaliar o modelo a partir de dados que ele nao usou na etapa de aprendizado? A importância está no que chamamos de capacidade de generalização.

A capacidade de generalização em aprendizado de máquina diz respeito ao quão bom o modelo é (ou o quão generalista ele é) para lidar com novos dados. No exemplo utilizado nessa aula, com os dados de treinamento a máquina aprenderia a prever se um cliente compraria ou não o produto do banco. Mas e para os novos clientes do banco ou mesmos os antigos mas com características diferentes (salário maior, por exemplo), será que o modelo ainda consegue prever bem a qual classe eles pertenceriam? Isso é a capacidade de generalização.

Quando avaliamos o modelo nos dados de teste, verificamos o desempenho do mesmo para novos dados. Então, temos uma noção se o processo de aprendizado foi adequado. Pode ser que, ao calcularmos o desempenho nos dados de treinamento, a máquina tenha aprendido muito bem. Mas, para os dados de teste, pode ser que não. Isso tem a ver com o que chamamos de sobreajuste (ou *overfitting*). Isso acontece quando o modelo fica tão especialista nos dados de treinamento que não consegue alcançar um bom desempenho nos dados de teste. Veja um exemplo na figura abaixo.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_overfitting_class.png' width="700">

Fonte: Retirado de *Hulsen, Tim, et al. "From big data to precision medicine." Frontiers in medicine, 34(6): 1-14, 2019.*

Outro problema que pode ser obtido na etapa de treinamento é o modelo não conseguir aprender bem a relação entre entrada e saída. Esse aspecto não está diretamente relacionado à identificação nos dados de teste. Isso diz mais respeito ao processo de treinamento em si. Uma causa poderia ser a adoção de um método inadequado para os dados. Ou o método pode ser adequado, mas nao foi treinado o suficiente. Veja um exemplo na figura abaixo.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_underfitting_class.png' width="700">

Fonte: Retirado de [ML | Underfitting and Overfitting](https://www.geeksforgeeks.org/underfitting-and-overfitting-in-machine-learning/)

# Medindo o desempenho do classificador

Uma vez treinado e testado o modelo, verificamos a qualidade do mesmo. Em problemas de classificação, as medidas de desempenho são baseadas quatro informações retiradas dos rótulos reais e das previsões feitas pelo modelo:

1. Verdadeiros Positivos (*True Positives* - TP): Amostras que pertencem à classe positiva e que foram corretamente classificadas como positivas.

2. Falsos Negativos (*False Negatives* - FN): Amostras que pertencem à classe positiva, mas que foram erroneamente classificadas como negativas.

3. Verdadeiros Negativos (*True Negatives* - TN): Amostras que pertencem à classe negativa e que foram corretamente classificadas como negativas.

4. Falsos Positivos (*False Positives* - FP): Amostras que pertencem à classe negativa, mas que foram erroneamente classificadas como positivas.

A classe positiva é representada como "1" ou "verdadeira", e a classe negativa é representada como "0" ou "falsa". Geralmente, para facilitar a visualização dos resultados, incluímos essas informações em uma matriz, chamada de Matriz de Confusão. Veja abaixo:

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_matriz_confusao.png'>

A matriz de confusão é, então, uma visualização tabular dos rótulos verdadeiros versus as previsões do modelo. A partir dela, derivamos diversas métricas de desempenho utilizadas para avaliador classificadores. Uma delas é a acurácia, a qual é dada pela proporção de previsões corretas feitas pelo modelo (tanto da classe positiva, quanto da negativa). Ou seja, é a razão entre o número de previsões corretas e o número total de previsões, ou ainda, a porcentagem de acertos.

Matematicamente, a acurácia é dada por

$$acuracia = \frac{TP + TN}{TP + TN + FP + FN}.$$

Veremos no decorrer da disciplina outras métricas de desempenho.